In [ ]:
"""
/shorts URL 유효성 병렬 체크 + 체크포인트 저장 + 재시작 지원
- 입력: youtube_max180_clean_with_flag.csv  (duration_seconds, shorts_tag_flag 포함 가정)
- 출력: youtube_max180_with_shorts_url.csv  (shorts_url_valid, is_shorts_* 추가)
"""

import os, time, re
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

INPUT  = "youtube_max180_clean_with_flag.csv"
OUTPUT = "youtube_max180_with_shorts_url.csv"
MAX_WORKERS = 20          # 동시 스레드 수 (10~32 권장)
DELAY = 0.03              # 요청 간 딜레이(차단 방지)
CHECKPOINT_EVERY = 200    # 이 개수마다 중간 저장
TIMEOUT = 6

def build_session():
    s = requests.Session()
    retries = Retry(
        total=3, backoff_factor=0.4,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET", "HEAD"],
    )
    s.headers.update({"User-Agent":"Mozilla/5.0 (compatible; ShortsURLChecker/1.0)"})
    s.mount("https://", HTTPAdapter(max_retries=retries))
    return s

session = build_session()

def extract_video_id(x: str):
    if not isinstance(x, str): return None
    if re.fullmatch(r"[A-Za-z0-9_-]{11}", x): return x
    m = re.search(r"(?:[?&]v=|/shorts/)([A-Za-z0-9_-]{11})", x)
    return m.group(1) if m else None

def check_shorts_url(video_id: str):
    """최종 URL이 /shorts/VIDEO_ID 형태로 유지되면 True, 아니면 False, 실패시 None"""
    try:
        url = f"https://www.youtube.com/shorts/{video_id}"
        r = session.get(url, timeout=TIMEOUT, allow_redirects=True)
        final_url = getattr(r, "url", "")
        if r.status_code == 200 and "/shorts/" in final_url:
            return True
        if "/watch" in final_url or "v=" in final_url:
            return False
        return False
    except Exception:
        return None

def ensure_cols(df: pd.DataFrame) -> pd.DataFrame:
    # videoId 확보
    if "videoId" not in df.columns:
        df["videoId"] = df["url"].apply(extract_video_id)
    else:
        df["videoId"] = df["videoId"].apply(extract_video_id)

    # 보조 플래그(없으면 생성)
    if "shorts_tag_flag" not in df.columns:
        df["shorts_tag_flag"] = 0
    if "duration_seconds" not in df.columns:
        df["duration_seconds"] = None
    return df

def main():
    # 입력 로드 (출력 파일이 이미 있으면 그걸 이어서 사용)
    if os.path.exists(OUTPUT):
        df = pd.read_csv(OUTPUT)
        print(f"[재시작] 기존 출력 파일 로드: {OUTPUT} (rows={len(df)})")
    else:
        df = pd.read_csv(INPUT)
        print(f"[시작] 입력 파일 로드: {INPUT} (rows={len(df)})")

    df = ensure_cols(df)
    if "shorts_url_valid" not in df.columns:
        df["shorts_url_valid"] = pd.NA

    # 처리 대상 인덱스(아직 값이 비어있는 행만)
    todo_idx = df.index[df["shorts_url_valid"].isna()].tolist()
    print(f"처리 대상: {len(todo_idx)}개")

    futures = {}
    processed = 0
    start = time.time()

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        for i in todo_idx:
            vid = df.at[i, "videoId"]
            if not isinstance(vid, str) or len(vid) != 11:
                df.at[i, "shorts_url_valid"] = None
                continue
            futures[ex.submit(check_shorts_url, vid)] = i
            # 가벼운 지연(전체 속도 제어)
            time.sleep(DELAY)

        for fut in as_completed(futures):
            i = futures[fut]
            try:
                df.at[i, "shorts_url_valid"] = fut.result()
            except Exception:
                df.at[i, "shorts_url_valid"] = None

            processed += 1
            if processed % CHECKPOINT_EVERY == 0:
                df.to_csv(OUTPUT, index=False)
                print(f"중간 저장: {processed}/{len(todo_idx)}")

    # 최종 저장
    df.to_csv(OUTPUT, index=False)
    elapsed = time.time() - start
    print(f"완료 저장: {OUTPUT} (경과 {elapsed:.1f}s)")

    # 최종 숏츠 라벨(보수/넓게)
    # 보수: 180초 이하 AND (해시태그 or shorts_url_valid)
    def to_bool(x): 
        if pd.isna(x): return False
        return bool(x)

    df["duration_le_180"] = pd.to_numeric(df["duration_seconds"], errors="coerce").fillna(10**9) <= 180
    df["is_shorts_conservative"] = df["duration_le_180"] & (df["shorts_tag_flag"].astype(bool) | df["shorts_url_valid"].apply(to_bool))
    df["is_shorts_broad"] = df["duration_le_180"] | df["shorts_tag_flag"].astype(bool) | df["shorts_url_valid"].apply(to_bool)

    df.to_csv(OUTPUT, index=False)
    print("요약:")
    print("shorts_url_valid =", df["shorts_url_valid"].value_counts(dropna=False).to_dict())
    print("is_shorts_conservative =", int(df["is_shorts_conservative"].sum()))
    print("is_shorts_broad =", int(df["is_shorts_broad"].sum()))

if __name__ == "__main__":
    main()


[시작] 입력 파일 로드: youtube_max180_clean_with_flag.csv (rows=2843)
처리 대상: 2843개
중간 저장: 200/2843
중간 저장: 400/2843
중간 저장: 600/2843
중간 저장: 800/2843
중간 저장: 1000/2843
중간 저장: 1200/2843
중간 저장: 1400/2843
중간 저장: 1600/2843
중간 저장: 1800/2843
중간 저장: 2000/2843
중간 저장: 2200/2843
중간 저장: 2400/2843
중간 저장: 2600/2843
중간 저장: 2800/2843
완료 저장: youtube_max180_with_shorts_url.csv (경과 193.3s)
요약:
shorts_url_valid = {True: 2783, False: 60}
is_shorts_conservative = 2783
is_shorts_broad = 2843


In [ ]:
import pandas as pd

# ===== 설정 =====
INPUT_FILE = "youtube_max180_with_shorts_url.csv"   # 원본 CSV
OUTPUT_FILE = "youtube_filtered_clean.csv"          # 저장할 CSV
# ================

# 0. CSV 불러오기
df = pd.read_csv(INPUT_FILE)

# 1. is_shorts_conservative 값이 False인 데이터 전부 삭제
df = df[df['is_shorts_conservative'] == True].copy()

# 2. 지정된 6개 컬럼 삭제
cols_to_drop = [
    'shorts_url_valid',
    'duration_le_180',
    'shorts_tag_flag',
    'is_shorts_conservative',
    'is_shorts_broad',
    'shortsHashtags'
]
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

# 3. category = 'Unknown' 데이터 삭제 (대소문자 구분 안 함)
if 'category' in df.columns:
    df = df[~df['category'].astype(str).str.lower().eq('unknown')].copy()

# 4. 결과 저장
df.to_csv(OUTPUT_FILE, index=False)

print(f"필터링 완료! 저장 경로: {OUTPUT_FILE}")
print(f"최종 데이터 행 수: {len(df)}")


필터링 완료! 저장 경로: youtube_filtered_clean.csv
최종 데이터 행 수: 2725


---
- 수치형

In [ ]:
# 로그 변환 + Min-Max 정규화를 적용하였습니다.
# Min-Max 정규화는 각 값이 전체에서 어느 정도 비율인지 알아보기 위해 데이터 범위를 0~1로 변환하였으며
# viewCount_norm 컬럼을 예로 들면 아래와 같은 방법으로 계산하였습니다.
# df['viewCount_norm'] = (df['viewCount'] - df['viewCount'].min()) / (df['viewCount'].max() - df['viewCount'].min())
# viewCount_log 컬럼은 조회수나 구독자 수처럼 편차가 큰 데이터에 대해 로그 변환으로 분포를 완화하기 위하여 
# df['viewCount_log'] = np.log1p(df['viewCount'])  # log(1+x)
# +1을 하는 이유: 조회수가 0인 경우에도 계산 가능하게 하기 위함
# 로그의 밑(base)는 **자연로그(e)**입니다.
# _log 컬럼은 로그 변환 값을 의미하고 _norm 컬럼은 로그 변환 후 Min-Max 정규화 값

# 이렇게 한 이유
# 조회수 데이터는 대개 소수의 영상이 매우 큰 값을 갖고 대부분은 작은 값을 가지는 치우친 분포(skewed distribution)를 보입니다.
# 로그 변환은 이 분포를 더 완만하게 만들어줍니다. (출력된 그래프 중 원본데이터의 그래프 볼것)
# 비율적 차이 반영: 로그 변환 후 값의 차이는 "몇 배 차이"를 나타내는 것과 비슷해집니다.
# 예: 조회수가 10배 늘면, 로그 값은 일정한 간격만큼 증가.
#| viewCount | viewCount\_log |
#| --------- | -------------- |
#| 0         | 0.00           |
#| 10        | 2.40           |
#| 100       | 4.61           |
#| 1,000     | 6.91           |
#| 10,000    | 9.21           |
#| 100,000   | 11.51          |

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

INPUT_FILE  = "./Utube_data/youtube_filtered_clean.csv"
OUTPUT_CSV  = "./Utube_data/youtube_filtered_clean_normalized_by_category.csv"
OUTPUT_DIR  = "./Utube_data/category_plots"

NUM_COLS = ["viewCount", "likeCount", "commentCount", "subscriberCount"]

def add_global_log_norm(df: pd.DataFrame) -> pd.DataFrame:
    for col in NUM_COLS:
        if col not in df.columns:
            continue
        log_col = f"{col}_log"
        df[log_col] = np.log1p(pd.to_numeric(df[col], errors="coerce").fillna(0))
        min_v = df[log_col].min()
        max_v = df[log_col].max()
        rng = max_v - min_v
        norm_col = f"{col}_norm"
        df[norm_col] = (df[log_col] - min_v) / rng if rng != 0 else 0
    return df

def add_category_log_norm(df: pd.DataFrame) -> pd.DataFrame:
    if "category" not in df.columns:
        raise ValueError("필수 컬럼 'category'가 없습니다.")
    for col in NUM_COLS:
        if col not in df.columns:
            continue
        logc = f"{col}_log_cat"
        normc = f"{col}_norm_cat"
        df[logc] = np.log1p(pd.to_numeric(df[col], errors="coerce").fillna(0))
        df[normc] = df.groupby("category")[logc].transform(
            lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0
        )
    return df

def plot_viewcount_distributions(df: pd.DataFrame, category: str, bins: int, out_png: str):
    sub = df[df["category"] == category].copy()
    if sub.empty:
        return
    plt.figure(figsize=(15, 4))

    plt.subplot(1, 3, 1)
    plt.hist(pd.to_numeric(sub["viewCount"], errors="coerce").fillna(0), bins=bins, edgecolor="black")
    plt.title(f"{category} - Original viewCount")

    plt.subplot(1, 3, 2)
    plt.hist(sub["viewCount_log_cat"], bins=bins, edgecolor="black", color="orange")
    plt.title(f"{category} - Log(1+viewCount)")

    plt.subplot(1, 3, 3)
    plt.hist(sub["viewCount_norm_cat"], bins=bins, edgecolor="black", color="green")
    plt.title(f"{category} - Normalized in Category (0~1)")

    plt.tight_layout()
    plt.savefig(out_png, dpi=150)
    plt.close()


# ------------------------
# 실행부
# ------------------------
df = pd.read_csv(INPUT_FILE)

# 전체 기준 로그/정규화
df = add_global_log_norm(df)

# 카테고리별 로그/정규화
df = add_category_log_norm(df)

# CSV 저장
df.to_csv(OUTPUT_CSV, index=False)
print(f"[CSV 저장] {OUTPUT_CSV} (rows={len(df)})")

# 출력 디렉터리 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 모든 카테고리별 그래프 저장
for category in df["category"].dropna().unique():
    safe_name = "".join(c if c.isalnum() or c in " _-" else "_" for c in category)
    out_path = os.path.join(OUTPUT_DIR, f"{safe_name}.png")
    plot_viewcount_distributions(df, category=category, bins=20, out_png=out_path)
    print(f"[PNG 저장] {out_path}")

[CSV 저장] ./Utube_data/youtube_filtered_clean_normalized_by_category.csv (rows=2725)
[PNG 저장] ./Utube_data/category_plots\Film _ Animation.png
[PNG 저장] ./Utube_data/category_plots\People _ Blogs.png
[PNG 저장] ./Utube_data/category_plots\Sports.png
[PNG 저장] ./Utube_data/category_plots\Comedy.png
[PNG 저장] ./Utube_data/category_plots\Entertainment.png
[PNG 저장] ./Utube_data/category_plots\Howto _ Style.png
[PNG 저장] ./Utube_data/category_plots\Autos _ Vehicles.png
[PNG 저장] ./Utube_data/category_plots\News _ Politics.png
[PNG 저장] ./Utube_data/category_plots\Gaming.png
[PNG 저장] ./Utube_data/category_plots\Pets _ Animals.png
[PNG 저장] ./Utube_data/category_plots\Music.png
[PNG 저장] ./Utube_data/category_plots\Science _ Technology.png


In [3]:
"""
category_plots/ 폴더에 있는 PNG 그래프들을 한 파일의 PDF로 병합합니다.
- 입력 폴더: category_plots
- 출력 파일: category_plots_all.pdf
"""

import os
import glob
from PIL import Image

INPUT_DIR = "./Utube_data/category_plots"
OUTPUT_PDF = "./Utube_data/category_plots_all.pdf"

def to_rgb(img: Image.Image) -> Image.Image:
    """PDF 저장을 위해 RGBA/LA/P 모드를 RGB로 변환"""
    if img.mode in ("RGBA", "LA", "P"):
        return img.convert("RGB")
    return img

def main():
    os.makedirs(INPUT_DIR, exist_ok=True)
    # PNG 파일들 수집 (이름순 정렬)
    png_paths = sorted(glob.glob(os.path.join(INPUT_DIR, "*.png")))
    if not png_paths:
        raise FileNotFoundError(f"PNG가 없습니다: {INPUT_DIR}/*.png")

    # 첫 장 + 나머지 페이지 분리
    first = to_rgb(Image.open(png_paths[0]))
    rest = [to_rgb(Image.open(p)) for p in png_paths[1:]]

    # PDF로 저장 (멀티페이지)
    first.save(
        OUTPUT_PDF,
        save_all=True,
        append_images=rest,
        resolution=150,
    )
    print(f"[완료] {len(png_paths)}개 이미지를 병합 → {OUTPUT_PDF}")

if __name__ == "__main__":
    main()


[완료] 12개 이미지를 병합 → ./Utube_data/category_plots_all.pdf


In [4]:
import pandas as pd

INPUT = "./Utube_data/youtube_filtered_clean.csv"
NUM_COLS = ["viewCount", "likeCount", "commentCount", "subscriberCount"]

df = pd.read_csv(INPUT)

# 1) category 값 진단
print("=== Category 값 개수 ===")
print(df['category'].value_counts(dropna=False))
print("\n유니크 카테고리 개수:", df['category'].nunique(dropna=True))

# 2) 지표별 결측 비율 확인
print("\n=== 지표별 결측 비율(%) ===")
for col in NUM_COLS:
    if col in df.columns:
        total = len(df)
        missing = df[col].isna().sum()
        print(f"{col}: {missing} / {total} ({(missing/total)*100:.2f}%)")
    else:
        print(f"{col}: ❌ 컬럼 없음")

# 3) 지표별 유효값 최소/최대 (데이터 분산 확인용)
print("\n=== 지표별 최소~최대값 ===")
for col in NUM_COLS:
    if col in df.columns:
        numeric_col = pd.to_numeric(df[col], errors="coerce")
        if numeric_col.notna().sum() > 0:
            print(f"{col}: min={numeric_col.min()}, max={numeric_col.max()}")
        else:
            print(f"{col}: ⚠️ 전부 NaN 또는 변환 불가")

=== Category 값 개수 ===
category
People & Blogs          992
Entertainment           447
Sports                  279
Comedy                  218
Film & Animation        189
Pets & Animals          171
Howto & Style           114
News & Politics         101
Autos & Vehicles         90
Science & Technology     44
Gaming                   42
Music                    38
Name: count, dtype: int64

유니크 카테고리 개수: 12

=== 지표별 결측 비율(%) ===
viewCount: 0 / 2725 (0.00%)
likeCount: 0 / 2725 (0.00%)
commentCount: 0 / 2725 (0.00%)
subscriberCount: 0 / 2725 (0.00%)

=== 지표별 최소~최대값 ===
viewCount: min=8186, max=124749205
likeCount: min=0, max=3527690
commentCount: min=0, max=31864
subscriberCount: min=10000, max=419000000


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 사용할 후보 폰트(환경에 있는 것부터 자동 선택)
FONT_CANDIDATES = [
    "Malgun Gothic",       # Windows
    "NanumGothic",         # Windows/Mac/Linux (나눔고딕 설치 시)
    "AppleGothic",         # macOS
    "Noto Sans CJK KR",    # Google Noto
    "Noto Sans KR",
    "DejaVu Sans",         # 최후의 보루(한글 일부 환경에서 지원)
]

def set_korean_font():
    available = {f.name for f in fm.fontManager.ttflist}
    for name in FONT_CANDIDATES:
        if name in available:
            plt.rcParams["font.family"] = name
            break
    # 마이너스 기호가 □ 로 안 나오게
    plt.rcParams["axes.unicode_minus"] = False

set_korean_font()


INPUT = "./Utube_data/youtube_filtered_clean.csv"
OUTPUT = "./Utube_data/youtube_filtered_clean_normalized_by_category.csv"

# ---- 0) CSV 읽기
df = pd.read_csv(INPUT)

# ---- 1) 정규화할 수치 컬럼
NUM_COLS = ["viewCount", "likeCount", "commentCount", "subscriberCount"]

# ---- 2) category 컬럼명 통일
if "category" not in df.columns:
    possible_cats = [c for c in df.columns if c.lower() == "category"]
    if possible_cats:
        df.rename(columns={possible_cats[0]: "category"}, inplace=True)
    else:
        raise ValueError("'category' 컬럼이 없습니다.")

# ---- 3) 카테고리 순서: viewCount 평균 기준 내림차순
category_order = (
    df.groupby("category")["viewCount"]
    .mean()
    .sort_values(ascending=False)
    .index
    .tolist()
)
df["category"] = pd.Categorical(df["category"], categories=category_order, ordered=True)

# ---- 4) 전체 + 카테고리별 로그 & 정규화
for col in NUM_COLS:
    if col not in df.columns:
        print(f"⚠️ {col} 없음, 스킵")
        continue

    col_numeric = pd.to_numeric(df[col], errors="coerce").fillna(0)

    # 전체 로그 변환 & Min-Max 정규화
    df[f"{col}_log"] = np.log1p(col_numeric)
    min_v, max_v = df[f"{col}_log"].min(), df[f"{col}_log"].max()
    df[f"{col}_norm"] = (df[f"{col}_log"] - min_v) / (max_v - min_v) if max_v != min_v else 0

    # 카테고리별 로그 변환 & Min-Max 정규화
    df[f"{col}_log_cat"] = np.log1p(col_numeric)
    df[f"{col}_norm_cat"] = df.groupby("category")[f"{col}_log_cat"].transform(
        lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0
    )

# ---- 5) CSV 저장
df.to_csv(OUTPUT, index=False)
print(f"저장 완료: {OUTPUT}")

# ---- 6) 전체 4개 지표를 한 이미지로 합친 분포 비교
fig, axes = plt.subplots(len(NUM_COLS), 2, figsize=(16, 4 * len(NUM_COLS)), sharey=False)

for idx, col in enumerate(NUM_COLS):
    log_col = f"{col}_log"
    norm_cat_col = f"{col}_norm_cat"
    if log_col not in df.columns or norm_cat_col not in df.columns:
        continue

    # 로그 변환 값
    df.boxplot(column=log_col, by="category", ax=axes[idx, 0], vert=True)
    axes[idx, 0].set_title(f"{col} - Log 변환 값")
    axes[idx, 0].set_xlabel("Category")
    axes[idx, 0].set_ylabel("값")
    axes[idx, 0].tick_params(axis='x', rotation=45)

    # 카테고리별 정규화 값
    df.boxplot(column=norm_cat_col, by="category", ax=axes[idx, 1], vert=True)
    axes[idx, 1].set_title(f"{col} - Category별 Min-Max 정규화 값")
    axes[idx, 1].set_xlabel("Category")
    axes[idx, 1].tick_params(axis='x', rotation=45)

plt.suptitle("카테고리별 로그 변환 vs 정규화 분포 비교 (정렬: viewCount 평균)", fontsize=18, y=1.02)
plt.tight_layout()
plt.savefig("category_norm_comparison_all_sorted.png", dpi=150, bbox_inches="tight")
plt.close()

print("그래프 저장 완료: category_norm_comparison_all_sorted.png")

C:\Users\YUN\AppData\Local\Temp\ipykernel_28208\1154245776.py:70: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f"{col}_norm_cat"] = df.groupby("category")[f"{col}_log_cat"].transform(
C:\Users\YUN\AppData\Local\Temp\ipykernel_28208\1154245776.py:70: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f"{col}_norm_cat"] = df.groupby("category")[f"{col}_log_cat"].transform(
C:\Users\YUN\AppData\Local\Temp\ipykernel_28208\1154245776.py:70: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observ

저장 완료: ./Utube_data/youtube_filtered_clean_normalized_by_category.csv
그래프 저장 완료: category_norm_comparison_all_sorted.png


In [6]:
# 카테고리(category) × 채널 티어(channelTier) 단위로 로그 변환 + 정규화
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

INPUT = "./Utube_data/youtube_filtered_clean.csv"
OUTPUT = "./Utube_data/youtube_filtered_clean_normalized_by_cat_tier.csv"

# ---- 0) CSV 읽기
df = pd.read_csv(INPUT)

# ---- 1) 조회수, 좋아요 수, 댓글 수, 구독자 수 4개 수치형 컬럼을 대상으로 작업
NUM_COLS = ["viewCount", "likeCount", "commentCount", "subscriberCount"]

# ---- 2) 컬럼명 통일
if "category" not in df.columns:
    possible_cats = [c for c in df.columns if c.lower() == "category"]
    if possible_cats:
        df.rename(columns={possible_cats[0]: "category"}, inplace=True)
    else:
        raise ValueError("'category' 컬럼이 없습니다.")

if "channelTier" not in df.columns:
    raise ValueError("'channelTier' 컬럼이 없습니다.")

# ---- 3) channelTier에서 숫자만 남기기
df["channelTier"] = df["channelTier"].astype(str).str.extract(r"(\d+)").fillna("").astype(str)

# ---- 4) 평균 조회수가 높은 순서대로 카테고리 순서를 고정
category_order = (
    df.groupby("category")["viewCount"]
    .mean()
    .sort_values(ascending=False)
    .index
    .tolist()
)
df["category"] = pd.Categorical(df["category"], categories=category_order, ordered=True)

# ---- 5) 전체 + 카테고리×티어별 로그 & 정규화
for col in NUM_COLS:
    if col not in df.columns:
        print(f"⚠️ {col} 없음, 스킵")
        continue

    col_numeric = pd.to_numeric(df[col], errors="coerce").fillna(0)

    # 전체 로그 변환 & Min-Max 정규화
    df[f"{col}_log"] = np.log1p(col_numeric)
    min_v, max_v = df[f"{col}_log"].min(), df[f"{col}_log"].max()
    df[f"{col}_norm"] = (df[f"{col}_log"] - min_v) / (max_v - min_v) if max_v != min_v else 0

    # 카테고리 × 티어별 로그 변환 & Min-Max 정규화
    df[f"{col}_log_cat_tier"] = np.log1p(col_numeric)
    df[f"{col}_norm_cat_tier"] = df.groupby(["category", "channelTier"])[f"{col}_log_cat_tier"].transform(
        lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0
    )

# ---- 6) CSV 저장
df.to_csv(OUTPUT, index=False)
print(f"저장 완료: {OUTPUT}")

# ---- 7) 시각화: 카테고리 × 티어 조합별 박스플롯
# 카테고리 순서와 티어 순서를 모두 고려하여 멀티인덱스 라벨 생성
df["cat_tier"] = df["category"].astype(str) + "_T" + df["channelTier"]

fig, axes = plt.subplots(len(NUM_COLS), 2, figsize=(20, 4 * len(NUM_COLS)), sharey=False)

for idx, col in enumerate(NUM_COLS):
    log_col = f"{col}_log"
    norm_cat_tier_col = f"{col}_norm_cat_tier"
    if log_col not in df.columns or norm_cat_tier_col not in df.columns:
        continue

    # 로그 변환 값
    df.boxplot(column=log_col, by="cat_tier", ax=axes[idx, 0], vert=True)
    axes[idx, 0].set_title(f"{col} - Log 변환 값")
    axes[idx, 0].set_xlabel("Category_Tier")
    axes[idx, 0].set_ylabel("값")
    axes[idx, 0].tick_params(axis='x', rotation=90)

    # 카테고리×티어별 정규화 값
    df.boxplot(column=norm_cat_tier_col, by="cat_tier", ax=axes[idx, 1], vert=True)
    axes[idx, 1].set_title(f"{col} - Category×Tier별 Min-Max 정규화 값")
    axes[idx, 1].set_xlabel("Category_Tier")
    axes[idx, 1].tick_params(axis='x', rotation=90)

plt.suptitle("카테고리×티어별 로그 변환 vs 정규화 분포 비교", fontsize=18, y=1.02)
plt.tight_layout()
plt.savefig("category_tier_norm_comparison_all.png", dpi=150, bbox_inches="tight")
plt.close()

print("그래프 저장 완료: category_tier_norm_comparison_all.png")

C:\Users\YUN\AppData\Local\Temp\ipykernel_28208\3884990670.py:54: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f"{col}_norm_cat_tier"] = df.groupby(["category", "channelTier"])[f"{col}_log_cat_tier"].transform(
C:\Users\YUN\AppData\Local\Temp\ipykernel_28208\3884990670.py:54: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f"{col}_norm_cat_tier"] = df.groupby(["category", "channelTier"])[f"{col}_log_cat_tier"].transform(
C:\Users\YUN\AppData\Local\Temp\ipykernel_28208\3884990670.py:54: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pa

저장 완료: ./Utube_data/youtube_filtered_clean_normalized_by_cat_tier.csv
그래프 저장 완료: category_tier_norm_comparison_all.png


In [7]:
# 조회수 상위 12개 조합

"""
카테고리×티어 조합 중 상위 N개만 골라
- 로그(viewCount_log) 분포
- 카테고리×티어 정규화(viewCount_norm_cat_tier) 분포
를 한 장의 이미지로 저장합니다.
입력: youtube_filtered_clean.csv  (정규화는 스크립트 내에서 계산)
출력: top_category_tier_norm_comparison.png
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ====== 설정 ======
INPUT = "./Utube_data/youtube_filtered_clean.csv"
OUTPUT_IMG = "./Utube_data/top_category_tier_norm_comparison.png"

N = 12                       # 상위 N개 조합만 표시
RANK_BY = "mean_view"        # "mean_view" (조회수 평균) 또는 "count" (샘플 수)
# ===================

# --- 데이터 로드
df = pd.read_csv(INPUT)

# --- 기본 컬럼 체크
if "category" not in df.columns:
    cand = [c for c in df.columns if c.lower() == "category"]
    if cand: df.rename(columns={cand[0]: "category"}, inplace=True)
    else: raise ValueError("'category' 컬럼이 없습니다.")
if "channelTier" not in df.columns:
    raise ValueError("'channelTier' 컬럼이 없습니다.")
for c in ["viewCount", "likeCount", "commentCount", "subscriberCount"]:
    if c not in df.columns:
        raise ValueError(f"'{c}' 컬럼이 없습니다.")

# --- 티어 숫자만 유지(문자가 섞여 있으면)
df["channelTier"] = df["channelTier"].astype(str).str.extract(r"(\d+)").fillna("").astype(str)

# --- 로그 & 정규화(카테고리×티어)
df["viewCount_num"] = pd.to_numeric(df["viewCount"], errors="coerce").fillna(0)
df["viewCount_log"] = np.log1p(df["viewCount_num"])

# 카테고리×티어별 정규화
df["viewCount_log_cat_tier"] = df["viewCount_log"]
df["viewCount_norm_cat_tier"] = df.groupby(["category", "channelTier"])["viewCount_log_cat_tier"] \
    .transform(lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0)

# --- 랭킹용 통계
g = df.groupby(["category", "channelTier"], as_index=False).agg(
    mean_view=("viewCount_num", "mean"),
    count=("viewCount_num", "size")
)

if RANK_BY == "count":
    top = g.sort_values("count", ascending=False).head(N)
else:
    top = g.sort_values("mean_view", ascending=False).head(N)

# --- 상위 N개 조합만 필터
top_keys = set(zip(top["category"], top["channelTier"]))
df_top = df[[ (c, t) in top_keys for c, t in zip(df["category"], df["channelTier"]) ]].copy()

# 보기 좋은 라벨
df_top["cat_tier"] = df_top["category"].astype(str) + "_T" + df_top["channelTier"]

# 상위 조합 순서 고정(선택한 랭킹 순서대로)
order = (top.assign(cat_tier=top["category"].astype(str) + "_T" + top["channelTier"])
              ["cat_tier"].tolist())
df_top["cat_tier"] = pd.Categorical(df_top["cat_tier"], categories=order, ordered=True)

# --- 그림: 상위 N개 조합만
fig, axes = plt.subplots(1, 2, figsize=(max(12, N*0.9), 6), sharey=False)

# (좌) 로그 분포
df_top.boxplot(column="viewCount_log", by="cat_tier", ax=axes[0], vert=True)
axes[0].set_title("viewCount - Log 변환 분포")
axes[0].set_xlabel("Category_Tier (상위 N)")
axes[0].set_ylabel("log(1 + viewCount)")
axes[0].tick_params(axis='x', rotation=75)

# (우) 카테고리×티어 정규화 분포
df_top.boxplot(column="viewCount_norm_cat_tier", by="cat_tier", ax=axes[1], vert=True)
axes[1].set_title("viewCount - Category×Tier 정규화(0~1) 분포")
axes[1].set_xlabel("Category_Tier (상위 N)")
axes[1].tick_params(axis='x', rotation=75)

plt.suptitle(f"상위 {N} 조합 (정렬 기준: {RANK_BY})", y=1.02, fontsize=14)
plt.tight_layout()
plt.savefig(OUTPUT_IMG, dpi=150, bbox_inches="tight")
plt.close()

print(f"[완료] 저장: {OUTPUT_IMG}")
print("선정된 상위 조합:")
print(top)

[완료] 저장: ./Utube_data/top_category_tier_norm_comparison.png
선정된 상위 조합:
                category channelTier     mean_view  count
26        People & Blogs           4  1.462225e+07     42
32  Science & Technology           4  1.216373e+07     10
35                Sports           4  1.133283e+07     15
14                Gaming           4  1.120319e+07      8
19                 Music           3  1.117783e+07      6
29        Pets & Animals           4  9.258458e+06     20
17         Howto & Style           4  9.112413e+06     16
8          Entertainment           4  5.178965e+06     81
20                 Music           4  4.396510e+06      5
31  Science & Technology           3  4.304239e+06     14
23       News & Politics           4  4.280149e+06     22
30  Science & Technology           2  3.911008e+06     20


In [ ]:
# 좋아요 수 상위 12개 조합

"""
likeCount 기준 상위 N개의 카테고리×티어 조합만 골라
(좌) 로그 변환 분포
(우) 카테고리×티어 내 정규화 분포
를 한 장에 저장
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# ====== 설정 ======
INPUT = "./Utube_data/youtube_filtered_clean.csv"
OUTPUT_IMG = "./Utube_data/top_category_tier_norm_comparison_likeCount.png"

N = 12                 # 상위 N개 조합
RANK_BY = "mean_like"  # "mean_like" 또는 "count"
# ===================

# --- 데이터 로드
df = pd.read_csv(INPUT)

# --- 기본 컬럼 정리
if "category" not in df.columns:
    cand = [c for c in df.columns if c.lower() == "category"]
    if cand:
        df.rename(columns={cand[0]: "category"}, inplace=True)
    else:
        raise ValueError("'category' 컬럼이 없습니다.")

if "channelTier" not in df.columns:
    raise ValueError("'channelTier' 컬럼이 없습니다.")

# 티어 숫자만 추출
df["channelTier"] = (
    df["channelTier"].astype(str).apply(lambda s: re.search(r"(\d+)", s))
    .apply(lambda m: m.group(1) if m else "")
    .astype(str)
)

# likeCount 숫자 변환
df["likeCount"] = pd.to_numeric(df["likeCount"], errors="coerce").fillna(0)

# 랭킹용 통계
g = df.groupby(["category", "channelTier"], as_index=False).agg(
    mean_like=("likeCount", "mean"),
    count=("likeCount", "size")
)

if RANK_BY == "count":
    top = g.sort_values("count", ascending=False).head(N)
else:
    top = g.sort_values("mean_like", ascending=False).head(N)

# 상위 N개 조합만 필터
top_keys = set(zip(top["category"], top["channelTier"]))
df_top = df[[ (c, t) in top_keys for c, t in zip(df["category"], df["channelTier"]) ]].copy()

# 라벨 & 순서
df_top["cat_tier"] = df_top["category"].astype(str) + "_T" + df_top["channelTier"]
order = (top.assign(cat_tier=top["category"].astype(str) + "_T" + top["channelTier"])
              ["cat_tier"].tolist())
df_top["cat_tier"] = pd.Categorical(df_top["cat_tier"], categories=order, ordered=True)

# 로그 변환 & 정규화
df_top["likeCount_log"] = np.log1p(df_top["likeCount"])
df_top["likeCount_norm_cat_tier"] = df_top.groupby(["category", "channelTier"])["likeCount_log"] \
    .transform(lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0)

# --- 플롯
fig, axes = plt.subplots(1, 2, figsize=(max(14, N*0.9), 6), sharey=False)

# (좌) 로그 변환 분포
df_top.boxplot(column="likeCount_log", by="cat_tier", ax=axes[0], vert=True)
axes[0].set_title("likeCount - Log 변환 분포")
axes[0].set_xlabel("Category_Tier (상위 N)")
axes[0].set_ylabel("log(1 + likeCount)")
axes[0].tick_params(axis='x', rotation=75)

# (우) 카테고리×티어 정규화 분포
df_top.boxplot(column="likeCount_norm_cat_tier", by="cat_tier", ax=axes[1], vert=True)
axes[1].set_title("likeCount - Category×Tier 정규화(0~1) 분포")
axes[1].set_xlabel("Category_Tier (상위 N)")
axes[1].tick_params(axis='x', rotation=75)

plt.suptitle(f"상위 {N} 조합 (정렬 기준: {RANK_BY})", y=1.02, fontsize=14)
plt.tight_layout()
plt.savefig(OUTPUT_IMG, dpi=150, bbox_inches="tight")
plt.close()

print(f"[완료] 저장: {OUTPUT_IMG}")
print("선정된 상위 조합:")
print(top)

[완료] 저장: ./Utube_data/top_category_tier_norm_comparison_likeCount.png
선정된 상위 조합:
                category channelTier      mean_like  count
26        People & Blogs           4  340233.214286     42
32  Science & Technology           4  333260.100000     10
35                Sports           4  296746.400000     15
14                Gaming           4  260315.500000      8
29        Pets & Animals           4  221522.100000     20
8          Entertainment           4  199987.432099     81
17         Howto & Style           4  170978.500000     16
31  Science & Technology           3  149474.285714     14
19                 Music           3  133392.666667      6
30  Science & Technology           2   88200.850000     20
5                 Comedy           4   76965.791667     24
7          Entertainment           3   70982.508571    175


In [9]:
# 댓글 수 상위 12개 조합

"""
commentCount 기준 상위 N개의 카테고리×티어 조합만 골라
(좌) 로그 변환 분포
(우) 카테고리×티어 내 정규화 분포
를 한 장에 저장
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# ====== 설정 ======
INPUT = "./Utube_data/youtube_filtered_clean.csv"
OUTPUT_IMG = "./Utube_data/top_category_tier_norm_comparison_commentCount.png"

N = 12                   # 상위 N개 조합
RANK_BY = "mean_comment"  # "mean_comment" 또는 "count"
# ===================

# --- 데이터 로드
df = pd.read_csv(INPUT)

# --- 기본 컬럼 정리
if "category" not in df.columns:
    cand = [c for c in df.columns if c.lower() == "category"]
    if cand:
        df.rename(columns={cand[0]: "category"}, inplace=True)
    else:
        raise ValueError("'category' 컬럼이 없습니다.")

if "channelTier" not in df.columns:
    raise ValueError("'channelTier' 컬럼이 없습니다.")

# 티어 숫자만 추출
df["channelTier"] = (
    df["channelTier"].astype(str).apply(lambda s: re.search(r"(\d+)", s))
    .apply(lambda m: m.group(1) if m else "")
    .astype(str)
)

# commentCount 숫자 변환
df["commentCount"] = pd.to_numeric(df["commentCount"], errors="coerce").fillna(0)

# 랭킹용 통계
g = df.groupby(["category", "channelTier"], as_index=False).agg(
    mean_comment=("commentCount", "mean"),
    count=("commentCount", "size")
)

if RANK_BY == "count":
    top = g.sort_values("count", ascending=False).head(N)
else:
    top = g.sort_values("mean_comment", ascending=False).head(N)

# 상위 N개 조합만 필터
top_keys = set(zip(top["category"], top["channelTier"]))
df_top = df[[ (c, t) in top_keys for c, t in zip(df["category"], df["channelTier"]) ]].copy()

# 라벨 & 순서
df_top["cat_tier"] = df_top["category"].astype(str) + "_T" + df_top["channelTier"]
order = (top.assign(cat_tier=top["category"].astype(str) + "_T" + top["channelTier"])
              ["cat_tier"].tolist())
df_top["cat_tier"] = pd.Categorical(df_top["cat_tier"], categories=order, ordered=True)

# 로그 변환 & 정규화
df_top["commentCount_log"] = np.log1p(df_top["commentCount"])
df_top["commentCount_norm_cat_tier"] = df_top.groupby(["category", "channelTier"])["commentCount_log"] \
    .transform(lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0)

# --- 플롯
fig, axes = plt.subplots(1, 2, figsize=(max(14, N*0.9), 6), sharey=False)

# (좌) 로그 변환 분포
df_top.boxplot(column="commentCount_log", by="cat_tier", ax=axes[0], vert=True)
axes[0].set_title("commentCount - Log 변환 분포")
axes[0].set_xlabel("Category_Tier (상위 N)")
axes[0].set_ylabel("log(1 + commentCount)")
axes[0].tick_params(axis='x', rotation=75)

# (우) 카테고리×티어 정규화 분포
df_top.boxplot(column="commentCount_norm_cat_tier", by="cat_tier", ax=axes[1], vert=True)
axes[1].set_title("commentCount - Category×Tier 정규화(0~1) 분포")
axes[1].set_xlabel("Category_Tier (상위 N)")
axes[1].tick_params(axis='x', rotation=75)

plt.suptitle(f"상위 {N} 조합 (정렬 기준: {RANK_BY})", y=1.02, fontsize=14)
plt.tight_layout()
plt.savefig(OUTPUT_IMG, dpi=150, bbox_inches="tight")
plt.close()

print(f"[완료] 저장: {OUTPUT_IMG}")
print("선정된 상위 조합:")
print(top)

[완료] 저장: ./Utube_data/top_category_tier_norm_comparison_commentCount.png
선정된 상위 조합:
                category channelTier  mean_comment  count
32  Science & Technology           4   1965.400000     10
29        Pets & Animals           4   1772.200000     20
26        People & Blogs           4   1508.357143     42
8          Entertainment           4   1502.691358     81
23       News & Politics           4   1365.500000     22
14                Gaming           4   1343.000000      8
31  Science & Technology           3   1302.071429     14
22       News & Politics           3   1030.207547     53
2       Autos & Vehicles           4    936.000000      5
19                 Music           3    912.833333      6
5                 Comedy           4    912.083333     24
12                Gaming           2    756.000000     19


In [10]:
# 구독자 수 상위 12개 조합

"""
subscriberCount 기준 상위 N개의 카테고리×티어 조합만 골라
(좌) 로그 변환 분포
(우) 카테고리×티어 내 정규화(0~1) 분포
를 한 장의 이미지로 저장
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# ====== 설정 ======
INPUT = "./Utube_data/youtube_filtered_clean.csv"
OUTPUT_IMG = "./Utube_data/top_category_tier_norm_comparison_subscriberCount.png"

N = 12                   # 상위 N개 조합
RANK_BY = "mean_sub"     # "mean_sub" 또는 "count"
# ===================

# --- 데이터 로드
df = pd.read_csv(INPUT)

# --- 기본 컬럼 정리
if "category" not in df.columns:
    cand = [c for c in df.columns if c.lower() == "category"]
    if cand:
        df.rename(columns={cand[0]: "category"}, inplace=True)
    else:
        raise ValueError("'category' 컬럼이 없습니다.")

if "channelTier" not in df.columns:
    raise ValueError("'channelTier' 컬럼이 없습니다.")

# 티어 숫자만 추출
df["channelTier"] = (
    df["channelTier"].astype(str).apply(lambda s: re.search(r"(\d+)", s))
    .apply(lambda m: m.group(1) if m else "")
    .astype(str)
)

# subscriberCount 숫자 변환
df["subscriberCount"] = pd.to_numeric(df["subscriberCount"], errors="coerce").fillna(0)

# 랭킹용 통계
g = df.groupby(["category", "channelTier"], as_index=False).agg(
    mean_sub=("subscriberCount", "mean"),
    count=("subscriberCount", "size")
)

if RANK_BY == "count":
    top = g.sort_values("count", ascending=False).head(N)
else:
    top = g.sort_values("mean_sub", ascending=False).head(N)

# 상위 N개 조합만 필터
top_keys = set(zip(top["category"], top["channelTier"]))
df_top = df[[ (c, t) in top_keys for c, t in zip(df["category"], df["channelTier"]) ]].copy()

# 라벨 & 순서 고정
df_top["cat_tier"] = df_top["category"].astype(str) + "_T" + df_top["channelTier"]
order = (top.assign(cat_tier=top["category"].astype(str) + "_T" + top["channelTier"])
              ["cat_tier"].tolist())
df_top["cat_tier"] = pd.Categorical(df_top["cat_tier"], categories=order, ordered=True)

# 로그 변환 & 정규화
df_top["subscriberCount_log"] = np.log1p(df_top["subscriberCount"])
df_top["subscriberCount_norm_cat_tier"] = df_top.groupby(["category", "channelTier"])["subscriberCount_log"] \
    .transform(lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0)

# --- 플롯
fig, axes = plt.subplots(1, 2, figsize=(max(14, N*0.9), 6), sharey=False)

# (좌) 로그 변환 분포
df_top.boxplot(column="subscriberCount_log", by="cat_tier", ax=axes[0], vert=True)
axes[0].set_title("subscriberCount - Log 변환 분포")
axes[0].set_xlabel("Category_Tier (상위 N)")
axes[0].set_ylabel("log(1 + subscriberCount)")
axes[0].tick_params(axis='x', rotation=75)

# (우) 카테고리×티어 정규화 분포
df_top.boxplot(column="subscriberCount_norm_cat_tier", by="cat_tier", ax=axes[1], vert=True)
axes[1].set_title("subscriberCount - Category×Tier 정규화(0~1) 분포")
axes[1].set_xlabel("Category_Tier (상위 N)")
axes[1].tick_params(axis='x', rotation=75)

plt.suptitle(f"상위 {N} 조합 (정렬 기준: {RANK_BY})", y=1.02, fontsize=14)
plt.tight_layout()
plt.savefig(OUTPUT_IMG, dpi=150, bbox_inches="tight")
plt.close()

print(f"[완료] 저장: {OUTPUT_IMG}")
print("선정된 상위 조합:")
print(top)

[완료] 저장: ./Utube_data/top_category_tier_norm_comparison_subscriberCount.png
선정된 상위 조합:
                category channelTier      mean_sub  count
32  Science & Technology           4  2.088400e+07     10
8          Entertainment           4  1.143519e+07     81
14                Gaming           4  7.191250e+06      8
20                 Music           4  4.602000e+06      5
35                Sports           4  4.538667e+06     15
26        People & Blogs           4  4.072619e+06     42
29        Pets & Animals           4  3.638000e+06     20
23       News & Politics           4  3.089545e+06     22
11      Film & Animation           4  2.718333e+06     12
17         Howto & Style           4  2.351875e+06     16
2       Autos & Vehicles           4  1.726000e+06      5
5                 Comedy           4  1.724583e+06     24


In [11]:
# 출력 결과 보시면 조회수, 좋아요 수, 댓글 수, 구독자 수 순위가 동일하진 않네요
# 상위 10개 조합 구성이 비슷비슷하긴 한데

"""
viewCount / likeCount / commentCount / subscriberCount
4개 지표의 상위 N 카테고리×티어 조합을 각각 뽑아
(좌) 로그 변환 분포 / (우) 카테고리×티어 정규화 분포
총 4행×2열 플롯으로 저장
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# ====== 설정 ======
INPUT = "./Utube_data/youtube_filtered_clean.csv"
OUTPUT_IMG = "./Utube_data/top_category_tier_norm_comparison_all.png"
N = 10   # 상위 N개 조합
RANK_BY = "mean"  # "mean" 또는 "count"
# ===================

metrics = ["viewCount", "likeCount", "commentCount", "subscriberCount"]

# --- 데이터 로드
df = pd.read_csv(INPUT)

# 컬럼 체크 & 변환
if "category" not in df.columns:
    cand = [c for c in df.columns if c.lower() == "category"]
    if cand:
        df.rename(columns={cand[0]: "category"}, inplace=True)
    else:
        raise ValueError("'category' 컬럼이 없습니다.")
if "channelTier" not in df.columns:
    raise ValueError("'channelTier' 컬럼이 없습니다.")

# 티어 숫자만 추출
df["channelTier"] = (
    df["channelTier"].astype(str).apply(lambda s: re.search(r"(\d+)", s))
    .apply(lambda m: m.group(1) if m else "")
    .astype(str)
)

# 숫자 변환
for m in metrics:
    df[m] = pd.to_numeric(df[m], errors="coerce").fillna(0)

# --- 플롯 준비
fig, axes = plt.subplots(len(metrics), 2, figsize=(16, 4*len(metrics)))

if len(metrics) == 1:
    axes = np.array([axes])  # 2D 보장

for row_idx, metric in enumerate(metrics):
    # 랭킹용 통계
    g = df.groupby(["category", "channelTier"], as_index=False).agg(
        mean_val=(metric, "mean"),
        count=(metric, "size")
    )

    if RANK_BY == "count":
        top = g.sort_values("count", ascending=False).head(N)
    else:
        top = g.sort_values("mean_val", ascending=False).head(N)

    # 상위 N개 조합만 필터
    top_keys = set(zip(top["category"], top["channelTier"]))
    df_top = df[[ (c, t) in top_keys for c, t in zip(df["category"], df["channelTier"]) ]].copy()

    # 라벨 & 순서 고정
    df_top["cat_tier"] = df_top["category"].astype(str) + "_T" + df_top["channelTier"]
    order = (top.assign(cat_tier=top["category"].astype(str) + "_T" + top["channelTier"])
                  ["cat_tier"].tolist())
    df_top["cat_tier"] = pd.Categorical(df_top["cat_tier"], categories=order, ordered=True)

    # 로그 변환 & 정규화
    log_col = f"{metric}_log"
    norm_col = f"{metric}_norm_cat_tier"
    df_top[log_col] = np.log1p(df_top[metric])
    df_top[norm_col] = df_top.groupby(["category", "channelTier"])[log_col] \
        .transform(lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0)

    # --- 플롯
    # (좌) 로그 변환
    df_top.boxplot(column=log_col, by="cat_tier", ax=axes[row_idx, 0], vert=True)
    axes[row_idx, 0].set_title(f"{metric} - Log 변환")
    axes[row_idx, 0].set_xlabel("Category_Tier (상위 N)")
    axes[row_idx, 0].set_ylabel(f"log(1 + {metric})")
    axes[row_idx, 0].tick_params(axis='x', rotation=75)

    # (우) 카테고리×티어 정규화
    df_top.boxplot(column=norm_col, by="cat_tier", ax=axes[row_idx, 1], vert=True)
    axes[row_idx, 1].set_title(f"{metric} - Category×Tier 정규화(0~1)")
    axes[row_idx, 1].set_xlabel("Category_Tier (상위 N)")
    axes[row_idx, 1].tick_params(axis='x', rotation=75)

plt.suptitle(f"상위 {N} 조합 비교 (정렬 기준: {RANK_BY})", y=1.02, fontsize=16)
plt.tight_layout()
plt.savefig(OUTPUT_IMG, dpi=150, bbox_inches="tight")
plt.close()

print(f"[완료] 저장: {OUTPUT_IMG}")


[완료] 저장: ./Utube_data/top_category_tier_norm_comparison_all.png
